In [1]:
# conda: holo37 xesmfTools

# Required to manipulate sys.path and access environment
# variables via os.environ.
import os, sys

# cd to the 'lib' directory in your notebooks directory
# $ export LIBROOT=`pwd`
# Start jupyter lab from any directory

# Still debugging the proper environment paths
# This uses full pathnames
if (os.environ.get('LIBROOT')):
    sys.path.append(os.environ.get('LIBROOT'))
else:
    # Detect use of this notebook in mybinder.org
    cDir = os.getcwd()
    if cDir == '/home/jovyan/gridTools':
        sys.path.append('/home/jovyan/gridTools/lib')

# This allows us to test relative paths
#sys.path.append('lib')

from sysinfo import SysInfo
info = SysInfo()
info.show(vList=['platform','python','esmf','esmpy','xgcm','xesmf',
                 'netcdf4','numpy','xarray',
                 'cartopy','matplotlib',
                 'jupyter_core','jupyterlab','notebook',
                 'dask'])
    
from gridutils import GridUtils
%matplotlib inline

System name                             : Linux
System name                             : mom6vm
Operating system release                : 5.8.0-50-generic
Operating system version                : #56~20.04.1-Ubuntu SMP Mon Apr 12 21:46:35 UTC 2021
Hardware identifier                     : x86_64
Active conda environment                : xesmfTools
Conda reported versions of software:
platform                                : linux-64
python                                  : 3.7.10
esmf                                    : 8.0.1
esmpy                                   : 8.0.1
xgcm                                    : not installed
xesmf                                   : 0.5.3
netcdf4                                 : 1.5.6
numpy                                   : 1.20.2
xarray                                  : 0.17.0
cartopy                                 : 0.19.0
matplotlib                              : 3.3.4
jupyter_core                            : 4.7.1
jupyterlab          

In [2]:
grd = GridUtils()
grd.setVerboseLevel('INFO')
grd.setDebugLevel(2)

New DEBUG level (2)


In [3]:
# Help (module)
help(GridUtils)

Help on class GridUtils in module gridutils:

class GridUtils(builtins.object)
 |  GridUtils(app={})
 |  
 |  Methods defined here:
 |  
 |  __init__(self, app={})
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addMessage(self, msg)
 |      Append new message to message buffer.
 |  
 |  adjustExternalLoggers(self)
 |      This adjusts some noisy loggers from other python modules.
 |  
 |  app(self)
 |      By calling this function, the user is requesting the application functionality of GridUtils().
 |      return the dashboard, but GridUtils() also has an internal pointer to the application.
 |  
 |  application(self, app={})
 |      Convienence function to attach application items to GridUtil so it can update certain portions of the application.
 |      
 |      app = {
 |          'messages': panel.widget.TextBox     # Generally a pointer to a panel widget for display of text
 |          'defaultFigureSize': (8,6)           # Default figure size to r

In [4]:
# Demonstrate a plotting error when you attempt to plot a grid without projection information
grd.clearGrid()
(figure, axes) = grd.plotGrid()

Please set the plot 'projection' parameter 'name'


In [5]:
# Make sure we erase any previous grid, grid parameters and plot parameters.
grd.clearGrid()

# Specify the grid parameters
# Reproduce an example grid 20x30 as demonstrated by Niki Zadah
# REF: https://github.com/nikizadehgfdl/grid_generation/blob/dev/jupynotebooks/regional_grid_spherical.ipynb
grd.setGridParameters({
    'projection': {
        'name': 'LambertConformalConic',
        'lon_0': 230.0,
        'lat_0': 40.0,
        'ellps': 'WGS84'
    },
    'dx': 20.0,
    'dxUnits': 'degrees',
    'dy': 30.0,
    'dyUnits': 'degrees',
    'tilt': 30,
    'gridResolution': 1.0,
    'gridMode': 2.0
})

# To set or update dictionary items in 'projection', you can use the dictionary format above with a direct assigment
# or use the subKey parameter as in below.  
#grd.setGridParameters({
#    'name': 'LambertConformalConic',
#    'lon_0': 230.0,
#    'lat_0': 40.0,
#    'ellps': 'WGS84'
#}, subKey='projection')

In [6]:
# This forms a grid in memory using the specified grid parameters
grd.makeGrid()

KeyError: 'lat_1'

In [ ]:
# Save the new grid to a netCDF file
grd.saveGrid(filename="configs/test/LCC_20x30.nc")

In [ ]:
# This prints out all the current grid parameters
# Note: for Lambert Conformal Conic grids, two additional projection parameters are computed.
#       First and second parallel for the grid (lat_1 and lat_2)
grd.showGridParameters()

In [ ]:
# Show the data summary from xarray for the grid
grd.grid

In [ ]:
# Define plot parameters so we can see what the grid looks like
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'NearsidePerspective',
            'lat_0': 40.0,
            'lon_0': 230.0
        },
        'extent': [-160.0 ,-100.0, 20.0, 60.0],
        'iLinewidth': 1.0,
        'jLinewidth': 1.0,
        'showGridCells': True,
        'title': "Nearside Perspective: 20x30 with 30 degree tilt",
        'iColor': 'k',
        'jColor': 'k'
    }
)

# Projection may be specified separately
grd.setPlotParameters(
    {
        'name': 'NearsidePerspective',
        'lat_0': 40.0,
        'lon_0': 230.0        
    }, subKey='projection'
)

In [ ]:
# When we call plotGrid() we have two python objects returned
# Figure object - you have control whether to show the 
#   figure or save the contents to an output file
# Axes object - you can further fine tune plot parameters, 
#   titles, axis, etc prior to the final plotting of the figure.
#   Some items may be configured via the figure object.
(figure, axes) = grd.plotGrid()

In [ ]:
# For display in jupyterlab, place the figure variable solo in a cell
figure

In [ ]:
# You can save the figure using the savefig() method on the
# figure object.  Many formats are possible.
figure.savefig('configs/test/LCC_20x30.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', transparent=False, bbox_inches=None, pad_inches=0.1)

In [ ]:
# Plot last grid also on a Mercator projection
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'Mercator',
            'lat_0': 40.0,
            'lon_0': 230.0
        },
        'extent': [-160.0 ,-100.0, 20.0, 60.0],
        'iLinewidth': 1.0,
        'jLinewidth': 1.0,
        'showGridCells': True,
        'title': "Mercator projection: 20x30 with 30 degree tilt",
        'iColor': 'k',
        'jColor': 'k'
    }
)

In [ ]:
(figure, axes) = grd.plotGrid()

In [ ]:
figure

In [ ]:
# Show in Lambert Conformal Conic
# It appears that the standard parallels are computed on the fly.
# (latGridCenter - (latSpan/2), latGridCenter + (latSpan/2))
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': { 
            'name': 'LambertConformalConic',
            'lon_0': 230.0,
            'lat_1': 25.0,
            'lat_2': 55.0
        },
        'extent': [-160.0 ,-100.0, 20.0, 60.0],
        'iLinewidth': 0.5,
        'jLinewidth': 1.0,
        'showGridCells': True,
        'title': 'Lambert Conformal Conic: 20x30 with 30 degree tilt',
        'iColor': 'r',
        'jColor': 'b'
    }
)

In [ ]:
(figure, axes) = grd.plotGrid()

In [ ]:
figure

In [ ]:
# Arctic Grid #6
# Demonstration of a North Polar Stereo projection grid
# Central longitude of 160.0
# True scale latitude is currently unknown
arctic6 = '/home/cermak/gridtools/gridTools/configs/Arctic6/ocean_hgrid.nc'
grd.clearGrid()
grd.openDataset(arctic6)
grd.readGrid()
grd.closeDataset()

In [ ]:
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'NorthPolarStereo',    
            'lon_0': 160.0
        },
        'extent': [-180, 180, 38, 90],
        'iLinewidth': 0.5,
        'jLinewidth': 1.0,
        'showGrid': True,
        'title': 'North Polar Stereo: Arctic6',
        'iColor': 'r',
        'jColor': 'b'
    }
)

In [ ]:
# In the next to calls to plotGrid() we store the plots to variables
# that are shown later.
(figureArctic, axes) = grd.plotGrid()

In [ ]:
# NEP7: North East Pacific Grid #7
# Demonstration of Lambert Conformal Conic projection
nep7 = '/home/cermak/gridtools/gridTools/configs/NEP7/ocean_hgrid.nc'
grd.clearGrid()
grd.openDataset(nep7)
grd.readGrid()
grd.closeDataset()

In [ ]:
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'LambertConformalConic',
            'lon_0': -91.0,
            'lat_1': 40.0,
            'lat_2': 60.0
        },
        'extent': [-180, 180, 10.5, 90],
        'iLinewidth': 0.5,
        'jLinewidth': 1.0,
        'showGrid': True,
        'title': 'Lambert Conformal Conic: NEP7',
        'iColor': 'r',
        'jColor': 'b'
    }
)

In [ ]:
(figureNEP, axes) = grd.plotGrid()

In [ ]:
# Show the above figures here instead.
# Only one figure per cell is supported.  Each figure is shown in separate cells.
figureArctic

In [ ]:
figureNEP